In [ ]:
import os
import pandas as pd
import sys
sys.path.insert(0, os.path.abspath('../lib'))
import config
from evaluate import get_results
import numpy as np

# Troubleshooting and visualisation
import IPython.display as ipd


# Task 1: Mosquito Event Detection (MED)
This code is complementary to the paper: HumBugDB: a large-scale acoustic mosquito dataset. Section B of `/docs/` gives detail on the meaning of the metadata fields that are present in the `csv` file `config.data_df`, while Section C describes in more detail the models used here as baselines.


This notebook provides the interface to partition data, extract features, train a BNN model in either PyTorch or Keras and evaluate its accuracy, precision-recall, confusion matrices and uncertainty metrics. Settings are specified in `config.py` and `config_pytorch.py` or `config_keras.py` which are located in `../lib`. Functions are imported from data and feature processing code in `../lib/feat_util.py`, model training in `../lib/runTorch.py` or `../lib/runKeras.py` and evaluation in `../lib/evaluate.py`.

### Data and feature configuration `config.py`
Specify the metadata (csv) location in `data_df`, with the location of the raw wave files in `data_dir`. The desired output for the features is set by `dir_out`. Model objects will be saved to `../models/PyTorch/` for PyTorch, or `../models/keras/` for Keras models.

#### Feat A
Feat A:  features extracted for [VGGish](https://github.com/harritaylor/torchvggish)'s model class, imported from `feat_vggish.py`. Edits can be made in `lib/PyTorch/vggish/{mel_features.py, vggish_input.py, vggish_params.py}`. A further discussion on feature transformations is given in Section B.3 of the [HumBugDB supplement](https://github.com/HumBug-Mosquito/HumBugDB/blob/devel/docs/NeurIPS_2021_HumBugDB_Supplement.pdf).


#### Feat B
Feat B uses log-mel features with `librosa`, configurable in `config.py` with the sample rate `rate`, to which data is re-sampled on loading, a window size `win_size` which determines the size of a training window (in number of _feature_ windows), `step_size`, which determines the step size taken by the window, `NFFT`, and `n_hop`, which are parameters for the core STFT transform upon which log-mel feature extraction is based. Finally, `n_feat` determines the number of mel-frequency bands.

In `librosa`, we can calculate the value of `win_size` to achieve a user's desired `duration` for a label as follows:

`win_size` = `duration` / `frame_duration`, where `frame_duration` = `n_hop`/`rate`. Librosa uses a default `hop_length` of `NFFT/4`.
The default values in `config.py` are optimised for `rate` = 8000 with  `win_size` = 30, `NFFT` = 2048, `n_hop` = `default`,  to achieve a label duration of $30 \times 2048/(4\times 8000) = 1.92$ (s).


### PyTorch `config_pytorch.py`
`config_pytorch.py` incldues settings to change the learning rate, `lr`, the number of maximum overrun steps for a particular training criteria `max_overrun`, the number of `epochs`, and the `batch_size`. The type of training method used can be written in `train_model.py`, which by default supports saving the best epoch models for either the training accuracy, `best_train_acc`, or validation accuracy, `best_val_acc`, if supplied to `train_model`.

### Keras `config_keras.py`
`tau = 1.0`,`lengthscale = 0.01`, are parameters used for $l2$ weight regularization supplied in lines 35-37 of `runKeras.py`. `dropout = 0.2` controls the dropout rate,`validation_split = 0.2`, is the fraction of data supplied as validation to the model callbacks in `model.fit`, line 105. `batch_size` controls the batch size, and `epochs`, set the number of epochs to train. Note the slight difference between the two packages in the way validation data is passed to the model training.

## Step 1: Choose Keras or PyTorch

In [ ]:
library = 'Keras'

if library == 'PyTorch':
    from PyTorch.runTorch import (train_model, load_model, evaluate_model, evaluate_model_aggregated, VGGishDropout,
    Resnet50DropoutFull, Resnet18DropoutFull, VGGishDropoutFeatB)
elif library == 'Keras':
    from tensorflow import keras
    from Keras.runKeras import train_model, load_model, evaluate_model, evaluate_model_aggregated
else:
    print('Library:', library, 'not supported. Please add your own code for support of that framework.')

## Step 2a: Data partitioning for feature extraction
Feel free to adjust the training data according to any criteria available from the metadata of the labels, which are stored in `config.data_df`. The code here imports all data that is not in testing to the training set (which can then be further split into validation).
Take extra care to make sure recordings from the same experimental group, as given in Table 2 of the main paper, and indicated in `df['country'], df['location_type']`. do not appear both in train and testing, resulting in overestimate of performance on evaluation. The assertion is given to perform a check to ensure no duplicates arise.

In [ ]:
df = pd.read_csv(config.data_df)

# To be kept: please do not edit the test set: these paths select test set A, test set B as described in the paper
idx_test_A = np.logical_and(df['country'] == 'Tanzania', df['location_type'] == 'field')
idx_test_B = np.logical_and(df['country'] == 'UK', df['location_type'] == 'culture')
idx_train = np.logical_not(np.logical_or(idx_test_A, idx_test_B))
df_test_A = df[idx_test_A]
df_test_B = df[idx_test_B]


df_train = df[idx_train]

# Modify by addition or sub-sampling of df_train here
# df_train ... 

# Assertion to check that train does NOT appear in test:
assert len(np.where(pd.concat([df_train,df_test_A,
                               df_test_B]).duplicated())[0]) == 0, 'Train dataframe contains overlap with Test A, Test B'

## Step 2b: Choose feature extraction pipeline

* `'FeatA'`: features extracted for [VGGish](https://github.com/harritaylor/torchvggish)'s model class, imported from `feat_vggish.py`. 

* `'FeatB'`: `librosa` features based on prior acoustic mosquito detection work [ECML-PKDD2021](https://github.com/HumBug-Mosquito/MozzBNN).

In [ ]:
feat_type = 'FeatB'

if feat_type == 'FeatA': #VGGish features (Feat. A)
    from feat_vggish import get_train_test_from_df, get_test_from_df, reshape_feat
#     reshape_test = False  # Keep in list form to perform feature aggregation
elif feat_type == 'FeatB': #log-mel-128 win 30 step 5 train, step 30 test, features (Feat. B)
    from feat_util import get_train_test_from_df, get_test_from_df
#     reshape_test = True # Reshape to match shape requirement of train for model prediction
else:
    print('Features:', feat_type, 'not defined. Please check spelling or add your own code for support of those f.')

In [ ]:
X_train, y_train, X_test_A, y_test_A, X_test_B, y_test_B = get_train_test_from_df(df_train, df_test_A, df_test_B, debug=True)

### Performance optimisations
When creating features from data with mixed sample rates, `librosa.load` uses re-sampling (if specified) This is a really time consuming process, which may be circumvented by re-sampling the data once and storing in a folder before feature extraction.

### Feature processing or loading
Note that these settings require at least 16 GB RAM to load into memory for ResNet-50 processing, as channels are replicated 3 times to match the pre-trained weights model (see more on StackExchange [here](https://stackoverflow.com/questions/62971655/how-can-i-change-number-of-channels-on-resnet-to-make-it-work-only-on-b-w-images)). To reduce the strain on memory, increase the `step_size` parameter in `config.py` to reduce the number of windows created by feature extraction. This reduces the overlap between samples. 

Alternatively, it is possible to use a non-pretrained architecture and change the tensor creation code in `build_dataloader()` from `runTorch.py` to remove `.repeat(1,3,1,1)` as there will be no need to copy over identical data over three channels. 

Note that once the tensors have been created, VRAM is not an issue due to the batching over the dataloaders (this code has been run on a GTX970 with 3.5GB available VRAM).

If memory issues still persist, for further debugging see [here](https://medium.com/@raghadalghonaim/memory-leakage-with-pytorch-23f15203faa4).

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

## Step 3: Model training

In [ ]:
# for Keras, the function call is of form train_model(X_train, y_train, X_val=None, y_val=None)
model = train_model(X_train, y_train, X_val, y_val)

In [ ]:
# for PyTorch, choose from model = [VGGishDropout(), VGGishDropoutFeatB(), Resnet50DropoutFull(),
# Resnet18DropoutFull()]. See models in runTorch.py
model = train_model(X_train, y_train, X_val, y_val, model=VGGishDropoutFeatB())

## Step 4: Model evaluation

In [ ]:
# Optional: laod test data and model only to avoid committing X_train to memory
# X_test_A, y_test_A, X_test_B, y_test_B = get_test_from_df(df_test_A, df_test_B, pickle_name='vggish_feat_test_for_resize.pickle', debug=True)
X_test_A, y_test_A, X_test_B, y_test_B = get_test_from_df(df_test_A, df_test_B)

### Optional: load model from fully trained models
Models are stored in `'../outputs/models/keras/'` for Keras, and `'../outputs/models/pytorch/'` for PyTorch. When loading PyTorch models, specify the model object as `model=ModelObject()`, where `ModelObject()` is defined in `runPyTorch.py`. By default, supported models are: `VGGishDropout()`, `VGGishDropoutFeatB()`, `Resnet50DropoutFull()`, `Resnet18DropoutFull()`

In [ ]:
# Example in PyTorch
path = '../outputs/models/reproducibility/MED_trained_models/'
model_name = 'neurips_2021_humbugdb_mozzbnnv2_A.hdf5'
model = load_model(path + model_name)

In [ ]:
# Example in Keras
full_path = '../outputs/models/keras/'
model_name = 'neurips_2021_humbugdb_mozzbnnv2_B.hdf5'
model = load_model(path + model_name)

Generate BNN samples. Run with `n_samples` = 1 for deterministic NN, `n` >= 30 for BNN. Calculate the predictive entropy (PE), mutual information (MI), and log probabilities. Also plot the ROC curve and confusion matrix. Outputs are saved to `config.plot_dir` with `filename`. The code automatically aggregates features over the appropriate output shape depending on the feature type defined at the start of the notebook.

### Test A evaluation

In [ ]:
if feat_type == 'FeatA':
    p, yt = evaluate_model_aggregated(model, X_test_A, y_test_A, 30)  # Aggregate windows from feature list (0.96->1.92 s)
    PE, MI, log_prob = get_results(p, yt, filename = feat_type + '_' + model_name +'_Test_A')
elif feat_type == 'FeatB':
    y_preds_all = evaluate_model(model, X_test_A, y_test_A, 30)  # Predict directly over feature windows (1.92 s)
    PE, MI, log_prob = get_results(y_preds_all, y_test_A, filename = feat_type + '_' + model_name +'_Test_A')

### Test B evaluation

In [ ]:
if feat_type == 'FeatA':
    p, yt = evaluate_model_aggregated(model, X_test_B, y_test_B, 30)  # Aggregate windows from feature list (0.96->1.92 s)
    PE, MI, log_prob = get_results(p, yt, filename = feat_type + '_' + model_name +'_Test_B')
elif feat_type == 'FeatB':
    y_preds_all = evaluate_model(model, X_test_B, y_test_B, 30)  # Predict directly over feature windows (1.92 s)
    PE, MI, log_prob = get_results(y_preds_all, y_test_B, filename = feat_type + '_' + model_name +'_Test_B')
    